In [1]:
import numpy as np
import pandas as pd
import seaborn as sns 
import matplotlib.pyplot as plt
from skimpy import clean_columns

import warnings
warnings.filterwarnings("ignore")
warnings.warn("this will not show")

%matplotlib inline
# %matplotlib notebook

plt.rcParams["figure.figsize"] = (10,6)
# plt.rcParams['figure.dpi'] = 100

sns.set_style("whitegrid")
pd.set_option('display.float_format', lambda x: '%.2f' % x)

pd.options.display.max_rows = 1000
pd.options.display.max_columns = 1000
from skimpy import clean_columns
import re

In [2]:
df= pd.read_csv("Haus.csv")

In [3]:
df=clean_columns(df)

In [4]:
df.drop("unnamed_0",axis=1,inplace=True)

In [5]:
df.isnull().sum()/df.shape[0]

kaufpreis          0.01
zimmer             0.00
wohnflache_ca      0.01
street             0.00
city               0.00
anbieter           0.02
wohnungslage       0.07
bezug              0.12
description        0.03
wohnanlage         0.06
baujahr            0.15
energy             0.29
energy_2           0.17
lage               0.04
price_&_kosten     1.00
baujahr_1          1.00
wohnflache         0.99
zustand            0.93
zustand_1          1.00
3                  1.00
personenaufzug     0.99
anbieter_1         1.00
2                  1.00
unnamed_24         0.99
personenaufzug_1   1.00
mindestpreis       1.00
stellplatz         1.00
1                  1.00
dtype: float64

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1926 entries, 0 to 1925
Data columns (total 28 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   kaufpreis         1903 non-null   object 
 1   zimmer            1926 non-null   object 
 2   wohnflache_ca     1905 non-null   object 
 3   street            1926 non-null   object 
 4   city              1926 non-null   object 
 5   anbieter          1895 non-null   object 
 6   wohnungslage      1799 non-null   object 
 7   bezug             1693 non-null   object 
 8   description       1872 non-null   object 
 9   wohnanlage        1817 non-null   object 
 10  baujahr           1631 non-null   object 
 11  energy            1376 non-null   object 
 12  energy_2          1589 non-null   object 
 13  lage              1857 non-null   object 
 14  price_&_kosten    2 non-null      object 
 15  baujahr_1         0 non-null      float64
 16  wohnflache        21 non-null     object 


### 1) Kaufpreis

In [7]:
df["Hausprice"]=df["kaufpreis"].str.strip("€\xa0").str.replace(".","").astype("float")
df.drop("kaufpreis",axis=1,inplace=True)

### 2) Zimmer

In [8]:
df["Room"]=df["zimmer"].str.strip(" ").str.replace(",",".").replace("k.A.","Unknown")

In [9]:
df.drop("zimmer",axis=1,inplace=True)

### 3) Wohnfläche

In [10]:
df["Area"]=df["wohnflache_ca"].str.strip(" m²").str.replace(",",".").astype("float")

In [11]:
df.drop("wohnflache_ca",axis=1,inplace=True)

### 4) Street

In [12]:
df["street"]=df["street"].apply(lambda x : np.nan if x=="Straße nicht freigegeben" else x)

### 5) City 

In [13]:
df["Post_Code"]=df["city"].str.split(" ").str[0]
df["District"]=df["city"].str.split(" ").str[2].str.strip("()")

In [14]:
list1=[]
for i in df["city"]:
    try:
        if i.split(" ")[2]=="-":
            list1.append(i.split("-")[1])
  
        elif i.split(" ")[1]=="/":
            list1.append(i.split("/")[1])

        else:
            list1.append(i.split(" ")[2])
    except:
        list1.append(np.nan)
        

In [15]:
df.drop("city",axis=1,inplace=True)

In [16]:
df["District"]=pd.DataFrame(list1)

### 6) Anbieter 

In [17]:
df["Offerer"]=df["anbieter"]

In [18]:
df.drop("anbieter",axis=1,inplace=True)

### 7) Wohnungslage

In [ ]:
df["wohnungslage"]=df["wohnungslage"].astype("str")

In [105]:
list2=[]
for i in df["wohnungslage"]:
    if "Untergeschoss" in i:
        list2.append("Untergeschoss")
        continue
    if "Dach" in i:
        try:
            list2.append(re.findall('[0-9]+',i)[0]+" "+"Dachgeschoss")
        except:
            list2.append(np.nan)
        continue
    elif "Erd" in i:
        list2.append("Erdgeschoss")
        continue
    elif "Geschoss" in i :
        list2.append(re.findall('[0-9]+',i)[0])
        continue
    elif i.isdigit():
        list2.append(i)
        continue
    else:
        list2.append(np.nan)

In [106]:
df["Floor"]=list2

In [107]:
df[["wohnungslage","Floor"]].head(100)

,wohnungslage,Floor
0,6,6
1,1,1
2,Wohnungslage6. GeschossBezugsofort,6
3,KategorieMaisonetteWohnungslage1. GeschossBezu...,1
4,Bezugsofort,NaN
5,KategorieApartmentWohnungslage1. Geschoss,1
6,nan,NaN
7,Wohnungslage4. Geschoss,4
8,KategoriePenthouseBezugsofort,NaN
9,nan,NaN


In [108]:
df.drop("wohnungslage",axis=1,inplace=True)

In [115]:
df["Floor"]=df["Floor"].fillna("O")

In [120]:
list3=[]
for i in df["Floor"]:
    if "Untergeschoss" in i:
        list3.append("Basemet")
        continue
    elif "Dach" in i:
        try:
            list3.append(re.findall('[0-9]+',i)[0]+" "+"Roof")
        except:
            list3.append(np.nan)
        continue
    elif "Erd" in i:
        list3.append("Ground")
        continue
    elif i=="O":
        list3.append(i)
    else:
        list3.append(i)

In [124]:
df["Floor"]=list3

In [128]:
df["Floor"]=df["Floor"].replace("O",np.nan)

### 8) Baujahr

In [20]:
df["Year"]=df["baujahr"].str.strip("")

In [21]:
df.drop("baujahr",axis=1,inplace=True)

### 9) Energy

In [22]:
df.drop("energy",axis=1,inplace=True)

In [23]:
df["energy_2"]

0       EnergieträgerFernwärmeHeizungsartZentralheizung
1                                      EnergieträgerGas
2       EnergieträgerFernwärmeHeizungsartZentralheizung
3                                      EnergieträgerGas
4             EnergieträgerGasHeizungsartZentralheizung
                             ...                       
1921          EnergieträgerGasHeizungsartZentralheizung
1922    EnergieträgerFernwärmeHeizungsartZentralheizung
1923    EnergieträgerFernwärmeHeizungsartZentralheizung
1924                          HeizungsartZentralheizung
1925                                                NaN
Name: energy_2, Length: 1926, dtype: object

In [24]:
df.drop("price_&_kosten",axis=1,inplace=True)

In [25]:
df.drop("baujahr_1",axis=1,inplace=True)

In [26]:
df.drop("wohnflache",axis=1,inplace=True)

### 10) Zustand

In [27]:
df["Status"]=df["zustand"].str.replace("Neubau","New").replace("Altbau","Old")

In [28]:
df.drop("zustand",axis=1,inplace=True)

In [29]:
df.drop(["zustand_1","3","personenaufzug","anbieter_1","2","unnamed_24","personenaufzug_1","mindestpreis","stellplatz","1"],axis=1,inplace=True)

### 11) Bezug

In [131]:
df["bezug"].value_counts(dropna=False)

NaN                                                        233
EnergieausweistypVerbrauchsausweis                         223
Bezugsofort                                                213
Derzeitige Nutzungvermietet                                185
Bezugnach Vereinbarung                                     133
Bezugab sofort                                              90
EnergieausweistypBedarfsausweis                             70
Bezugnach Absprache                                         64
GebäudetypWohngebäude                                       60
Bezugvermietet                                              48
WohnungslageErdgeschoss                                     48
Wohnungslage2. Geschoss                                     31
Wohnungslage1. Geschoss                                     30
Bezug2023                                                   21
Wohnungslage3. Geschoss                                     20
BezugNach Vereinbarung                                 

In [148]:
df["bezug"]=df["bezug"].fillna("NaN")

In [260]:
list4=[]
for i in df["bezug"]:
    try:
        if "Vereinbarung" in i:
            list4.append("Nach Vereinbarung")
        elif "Absprache" in i:
            list4.append("Nach Vereinbarung")
        elif "Anfrage" in i:
            list4.append("Nach Vereinbarung")
        elif "frei" in i:
            list4.append("Sofort")
        elif "sofort"  in i:
            list4.append("Sofort")
        elif len(re.findall('[0-9]+',i)) > 0:
            if len(re.findall('[0-9]+',i)) == 1   and len(re.findall('[0-9]+',i)[0]) > 2:
                list4.append(re.findall('[0-9]+',i)[0])
            elif len(re.findall('[0-9]+',i)) == 2 and len(re.findall('[0-9]+',i)[1]) > 2:
                list4.append(re.findall('[0-9]+',i)[1])
            elif len(re.findall('[0-9]+',i)) == 3 and len(re.findall('[0-9]+',i)[2]) > 2:
                list4.append(".".join(re.findall('[0-9]+',i)))
            else:
                list4.append(np.nan)
            continue
        else :
            list4.append(np.nan)
       
    except:
        list4.append(np.nan)
    
        
       

In [261]:
df["Own"]=list4

In [263]:
df.drop("bezug",axis=1,inplace=True)

In [264]:
df.columns

Index(['street', 'description', 'wohnanlage', 'energy_2', 'lage', 'Hausprice',
       'Room', 'Area', 'Post_Code', 'District', 'Offerer', 'Year', 'Status',
       'Floor', 'Own'],
      dtype='object')

In [270]:
df1= pd.read_csv("Haus.csv")

In [276]:
df1=clean_columns(df1)

In [281]:
df1

,unnamed_0,kaufpreis,zimmer,wohnflache_ca,street,city,anbieter,wohnungslage,bezug,description,wohnanlage,baujahr,energy,energy_2,lage,price_&_kosten,baujahr_1,wohnflache,zustand,zustand_1,3,personenaufzug,anbieter_1,2,unnamed_24,personenaufzug_1,mindestpreis,stellplatz,1
0,0,225.000 €,2,"56,90 m²",Straße nicht freigegeben,81669 München (Au-Haidhausen),Gewerblicher,6,Bezugsofort,Bad mit Wanne Balkon Einbauküche Böden: Fliese...,Baujahr: 1973 Personenaufzug,1973,EnergieausweistypVerbrauchsausweisGebäudetypWo...,EnergieträgerFernwärmeHeizungsartZentralheizung,"HAIDHAUSEN - S-BahnnäheKneipen, Kultur, Kinder...","HAIDHAUSEN - S-BahnnäheKneipen, Kultur, Kinder...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,799.000 €,4,95 m²,Straße nicht freigegeben,81825 München (Trudering-Riem),Gewerblicher,1,Bezugnach Vereinbarung,"Bad mit Fenster, Wanne und Dusche, Gäste-WC Ba...",Baujahr: 1990 Zustand: Altbau (bis 1945) 1 Ste...,1990,EnergieausweistypVerbrauchsausweisGebäudetypWo...,EnergieträgerGas,TruderingDer beliebte Stadtteil Trudering im M...,TruderingDer beliebte Stadtteil Trudering im M...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,225.000 €,2,"56,90 m²",Straße nicht freigegeben,81669 München (Au-Haidhausen),Gewerblicher,Wohnungslage6. GeschossBezugsofort,Bezugsofort,Bad mit Wanne Balkon Einbauküche Böden: Fliese...,Baujahr: 1973 Personenaufzug,1973,EnergieausweistypVerbrauchsausweisGebäudetypWo...,EnergieträgerFernwärmeHeizungsartZentralheizung,"HAIDHAUSEN - S-BahnnäheKneipen, Kultur, Kinder...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,799.000 €,4,95 m²,Straße nicht freigegeben,81825 München (Trudering-Riem),Gewerblicher,KategorieMaisonetteWohnungslage1. GeschossBezu...,Bezugnach Vereinbarung,"Bad mit Fenster, Wanne und Dusche, Gäste-WC Ba...",Baujahr: 1990 Zustand: Altbau (bis 1945) 1 Ste...,1990,EnergieausweistypVerbrauchsausweisGebäudetypWo...,EnergieträgerGas,TruderingDer beliebte Stadtteil Trudering im M...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,269.000 €,1,27 m²,Straße nicht freigegeben,81373 München (Sendling),Gewerblicher,Bezugsofort,EnergieausweistypVerbrauchsausweis,Baujahr: 1976 Personenaufzug,NaN,NaN,EnergieausweistypVerbrauchsausweisGebäudetypWo...,EnergieträgerGasHeizungsartZentralheizung,Der Stadtteil München - Sendling befindet sich...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1921,1921,750.000 €,3,"83,50 m²",Osterwaldstr. 57,80805 München (Schwabing-Freimann),NaN,Wohnungslage1. UntergeschossBezugnach Vereinba...,EnergieträgerGas,Einbauküche Zustand: renoviert / saniert Weite...,Baujahr: 1962 Personenaufzug,1962,NaN,EnergieträgerGasHeizungsartZentralheizung,"Schwabing-Freimann gilt als die ""Grüne Insel"" ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1922,1922,379.000 €,2,"50,27 m²",Straße nicht freigegeben,81673 München (Berg am Laim),Gewerblicher,Wohnungslage2. GeschossBezugab sofort,Bezugab sofort,Bad mit Dusche Einbauküche Zustand: gepflegt,Baujahr: 1962,1962,EnergieausweistypVerbrauchsausweisGebäudetypWo...,EnergieträgerFernwärmeHeizungsartZentralheizung,Die Immobilie befindet sich in ruhiger und grü...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1923,1923,1.250.000 €,4,105 m²,Straße nicht freigegeben,80796 München (Schwabing-West),Gewerblicher,Wohnungslage5. Geschoss,EnergieausweistypVerbrauchsausweis,Gäste-WC Einbauküche Ausstattung: neuwertig We...,Baujahr: 1955 Personenaufzug,1955,EnergieausweistypVerbrauchsausweisGebäudetypWo...,EnergieträgerFernwärmeHeizungsartZentralheizung,München-Schwabing - Ein weit über die Grenzen ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1924,1924,945.000 €,3,80 m²,Straße nicht freigegeben,81929 München (Bogenhausen),Gewerblicher,KategorieApartmentWohnungslage4. Geschoss (Dac...,Wohnungslage4. Geschoss (Dachgeschoss),"Bad mit Wanne, Gäste-WC Balk

In [282]:
df1.drop(["zustand_1","3","personenaufzug","anbieter_1","2","unnamed_24","personenaufzug_1","mindestpreis","stellplatz","1","unnamed_0"],axis=1,inplace=True)

In [331]:
df1

,kaufpreis,zimmer,wohnflache_ca,street,city,anbieter,wohnungslage,bezug,description,wohnanlage,baujahr,energy,energy_2,lage,price_&_kosten,baujahr_1,wohnflache,zustand
0,225.000 €,2,"56,90 m²",Straße nicht freigegeben,81669 München (Au-Haidhausen),Gewerblicher,6,Bezugsofort,Bad mit Wanne Balkon Einbauküche Böden: Fliese...,Baujahr: 1973 Personenaufzug,1973,EnergieausweistypVerbrauchsausweisGebäudetypWo...,EnergieträgerFernwärmeHeizungsartZentralheizung,"HAIDHAUSEN - S-BahnnäheKneipen, Kultur, Kinder...","HAIDHAUSEN - S-BahnnäheKneipen, Kultur, Kinder...",NaN,NaN,NaN
1,799.000 €,4,95 m²,Straße nicht freigegeben,81825 München (Trudering-Riem),Gewerblicher,1,Bezugnach Vereinbarung,"Bad mit Fenster, Wanne und Dusche, Gäste-WC Ba...",Baujahr: 1990 Zustand: Altbau (bis 1945) 1 Ste...,1990,EnergieausweistypVerbrauchsausweisGebäudetypWo...,EnergieträgerGas,TruderingDer beliebte Stadtteil Trudering im M...,TruderingDer beliebte Stadtteil Trudering im M...,NaN,NaN,NaN
2,225.000 €,2,"56,90 m²",Straße nicht freigegeben,81669 München (Au-Haidhausen),Gewerblicher,Wohnungslage6. GeschossBezugsofort,Bezugsofort,Bad mit Wanne Balkon Einbauküche Böden: Fliese...,Baujahr: 1973 Personenaufzug,1973,EnergieausweistypVerbrauchsausweisGebäudetypWo...,EnergieträgerFernwärmeHeizungsartZentralheizung,"HAIDHAUSEN - S-BahnnäheKneipen, Kultur, Kinder...",NaN,NaN,NaN,NaN
3,799.000 €,4,95 m²,Straße nicht freigegeben,81825 München (Trudering-Riem),Gewerblicher,KategorieMaisonetteWohnungslage1. GeschossBezu...,Bezugnach Vereinbarung,"Bad mit Fenster, Wanne und Dusche, Gäste-WC Ba...",Baujahr: 1990 Zustand: Altbau (bis 1945) 1 Ste...,1990,EnergieausweistypVerbrauchsausweisGebäudetypWo...,EnergieträgerGas,TruderingDer beliebte Stadtteil Trudering im M...,NaN,NaN,NaN,NaN
4,269.000 €,1,27 m²,Straße nicht freigegeben,81373 München (Sendling),Gewerblicher,Bezugsofort,EnergieausweistypVerbrauchsausweis,Baujahr: 1976 Personenaufzug,NaN,NaN,EnergieausweistypVerbrauchsausweisGebäudetypWo...,EnergieträgerGasHeizungsartZentralheizung,Der Stadtteil München - Sendling befindet sich...,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1921,750.000 €,3,"83,50 m²",Osterwaldstr. 57,80805 München (Schwabing-Freimann),NaN,Wohnungslage1. UntergeschossBezugnach Vereinba...,EnergieträgerGas,Einbauküche Zustand: renoviert / saniert Weite...,Baujahr: 1962 Personenaufzug,1962,NaN,EnergieträgerGasHeizungsartZentralheizung,"Schwabing-Freimann gilt als die ""Grüne Insel"" ...",NaN,NaN,NaN,NaN
1922,379.000 €,2,"50,27 m²",Straße nicht freigegeben,81673 München (Berg am Laim),Gewerblicher,Wohnungslage2. GeschossBezugab sofort,Bezugab sofort,Bad mit Dusche Einbauküche Zustand: gepflegt,Baujahr: 1962,1962,EnergieausweistypVerbrauchsausweisGebäudetypWo...,EnergieträgerFernwärmeHeizungsartZentralheizung,Die Immobilie befindet sich in ruhiger und grü...,NaN,NaN,NaN,NaN
1923,1.250.000 €,4,105 m²,Straße nicht freigegeben,80796 München (Schwabing-West),Gewerblicher,Wohnungslage5. Geschoss,EnergieausweistypVerbrauchsausweis,Gäste-WC Einbauküche Ausstattung: neuwertig We...,Baujahr: 1955 Personenaufzug,1955,EnergieausweistypVerbrauchsausweisGebäudetypWo...,EnergieträgerFernwärmeHeizungsartZentralheizung,München-Schwabing - Ein weit über die Grenzen ...,NaN,NaN,NaN,NaN
1924,945.000 €,3,80 m²,Straße nicht freigegeben,81929 München (Bogenhausen),Gewerblicher,KategorieApartmentWohnungslage4. Geschoss (Dac...,Wohnungslage4. Geschoss (Dachgeschoss),"Bad mit Wanne, Gäste-WC Balkon, Wintergarten E...",Baujahr: 1985 Personenaufzug 1 Stellplatz: Tie...,1985,EnergieausweistypVerbrauchsausweisGebäudetypWo...,HeizungsartZentralheizung,Der Zamilapark im Stadtteil Bogenhausen liegt ...,NaN,NaN,NaN,NaN


In [336]:
C=pd.concat([df.iloc[:,0],df1.iloc[:,3]],axis=1)

In [337]:
C=pd.concat([df.iloc[:,5],df1.iloc[:,0],df.iloc[:,6],df1.iloc[:,1],df.iloc[:,7],df1.iloc[:,2],df.iloc[:,8],df.iloc[:,9],df1.iloc[:,4],df.iloc[:,10],df1.iloc[:,5],df1.iloc[:,6],df.iloc[:,11],df1.iloc[:,7],df.iloc[:,12],df1.iloc[:,10],df.iloc[:,13],df1.iloc[:,4],df.iloc[:,14]],axis=1)

In [339]:
df.to_csv("Haus1.csv")